In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import xgboost as xgb
import optuna
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor, early_stopping
from xgboost import XGBRegressor

from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [4]:
df = pd.read_csv("trainable_df.csv")

In [5]:
df

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_Growth1,SCALER_Money_Handling_Level1,SCALER_Growth,SCALER_Money_Per_Head,SCALER_Money_Handling_Level,SCALER_Credit_by_Score,SCALER_Feedback1,SCALER_Determinstic,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_Health_Age_Interaction,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Marital_Status_Single,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount,SCALER_Number_of_Dependents,Premium Amount
0,4,-0.389132,-0.974138,-0.254990,-0.175976,-0.322796,-0.067034,-0.494442,-1.105691,-0.402072,-0.064163,28.5,24.535366,1.0,4.862885,-3.0,0.00,-0.117824,0.117824,-0.727595,-0.348706,0.000000,-0.598296,0.333333,0.7,-0.423860,-0.012354,-1.469576e-15,1.2,0.570401,0.000000,0.00,0.571429,-0.739699,0.00,0.726836,-0.497239,-0.500000,-0.683947,-0.333333,0.4,0.0,1.0,0.222222,0.0,-0.956522,-2.155882,-0.5,0.000000,2869.0
1,2,0.201502,0.413793,0.047060,0.112611,0.471303,-0.044334,0.383426,0.546341,0.231635,0.218843,-1.0,0.000000,0.0,0.000000,0.0,0.00,-0.515647,0.515647,0.173913,-0.525684,0.000000,-0.386547,0.333333,0.5,-0.266832,-0.383720,0.000000e+00,0.0,0.000000,-1.175476,-2.50,0.000000,0.458999,1.50,-0.273164,-0.284248,0.833333,-0.683947,-0.333333,0.4,0.0,1.0,0.333333,0.0,-0.086957,-0.783022,0.5,0.000000,1483.0
2,4,0.035582,-0.185345,0.867975,0.120111,-0.264470,-0.134425,0.009334,0.094309,0.695881,0.519340,-1.0,0.000000,0.0,0.000000,0.0,0.50,1.273265,-1.273265,0.912156,-0.421822,0.000000,0.182425,0.333333,0.6,2.630225,0.139051,-7.347881e-16,0.6,0.000000,0.000000,0.00,0.000000,-0.557755,0.00,-0.273164,1.081276,-0.500000,-0.683947,-0.333333,0.4,0.0,1.0,0.333333,0.0,-0.782609,0.000000,0.5,0.000000,567.0
3,3,3.210158,-0.995690,3.341497,5.392410,2.470710,2.640963,1.830458,-0.517073,1.164823,6.154136,-1.0,0.000000,0.0,0.000000,0.0,-0.25,-0.777782,0.777782,-0.732032,-0.908511,1.000000,-0.836041,0.666667,0.9,-0.638249,-0.279502,0.000000e+00,0.0,0.575111,-0.818739,-0.75,0.000000,-1.483870,-2.75,0.802161,-0.887272,0.500000,-0.683947,-0.333333,0.4,0.0,1.0,-0.666667,0.0,-0.869565,-0.133238,0.0,0.000000,765.0
4,3,0.419224,0.000000,0.552737,0.432616,0.304591,1.249722,0.466505,0.234146,-0.050167,1.293381,0.0,-1.000000,-1.0,-1.000000,1.0,-0.50,-0.243621,0.243621,-0.527063,-0.149231,0.753709,-0.600057,-0.333333,-0.1,-0.464725,-0.300577,-1.469576e-15,1.2,0.570401,-0.818739,-0.75,0.571429,0.032506,0.00,0.726836,-0.161752,0.500000,-0.683947,-0.333333,0.4,0.0,1.0,-0.333333,1.0,-0.869565,0.216978,-0.5,0.000000,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4,0.395494,-1.245690,0.529025,1.349580,0.286173,1.211067,-0.097738,-0.705691,0.400539,0.182273,-1.0,0.000000,0.0,0.000000,0.0,0.00,-0.576268,0.576268,-0.509317,-0.814398,0.753709,-0.246

#
---
#

# Just Checking!

In [6]:
def return_splits(ddf, feature_name, target_name):
    return [ddf[ddf[feature_name] == i][target_name] for i in ddf[feature_name].unique()]

def give_stats_analysis(df, target_column_name):
    ddf = df.copy()
    ddf = ddf.dropna()

    features = []
    tests = []
    stats = []
    pvals = []
    verdict = []
    count = 0

    target = ddf[target_column_name]
    for i in ddf.columns:
        features.append(i)
        feature = ddf[i]
        
        if (feature.dtype == "O" and (target.dtype == "float" or target.dtype == "int")) or (target.dtype == "O" and (feature.dtype == "float" or feature.dtype == "int")):
            stat, pval, *_ = kruskal(*return_splits(ddf, feature.name, target.name))
            tests.append("Kruskal-Wallis")
            stats.append(stat)
            pvals.append(pval)
            
        
        elif (feature.dtype == "float" or feature.dtype == "int") and (target.dtype == "float" or target.dtype == "int"):
            stat, pval, *_ = spearmanr(feature, target)
            tests.append("SpearmanR")
            stats.append(stat)
            pvals.append(pval)

        elif feature.dtype == "O" and target.dtype == "O":
            stat, pval, *_ = chi2_contingency(pd.crosstab(feature, target))
            tests.append("Chi-Square")
            stats.append(stat)
            pvals.append(pval)
        
        else:
            tests.append(np.nan)
            stats.append(np.nan)
            pvals.append(np.nan)
        
        if pval <= 0.05:
            verdict.append("There is Relationship")
        else:
            verdict.append("There is NO Relationship")

        print(f"{feature.name} ■■■ {target_column_name}".ljust(50, "-")+"✅")
    
    return pd.DataFrame({
        "Feature" : features,
        "Target" : [target_column_name]*ddf.shape[1],
        "Statistic Test" : tests,
        "Test Statistic" : stats,
        "P-Value" : pvals,
        "Verdict" : verdict
    }).sort_values(by="P-Value")

In [7]:
give_stats_analysis(df, "Premium Amount")

Health Conscious Level ■■■ Premium Amount---------✅
SCALER_Annual_Income ■■■ Premium Amount-----------✅
SCALER_Credit_Score ■■■ Premium Amount------------✅
SCALER_Growth1 ■■■ Premium Amount-----------------✅
SCALER_Money_Handling_Level1 ■■■ Premium Amount---✅
SCALER_Growth ■■■ Premium Amount------------------✅
SCALER_Money_Per_Head ■■■ Premium Amount----------✅
SCALER_Money_Handling_Level ■■■ Premium Amount----✅
SCALER_Credit_by_Score ■■■ Premium Amount---------✅
SCALER_Feedback1 ■■■ Premium Amount---------------✅
SCALER_Determinstic ■■■ Premium Amount------------✅
SCALER_Previous_Claims_MEDIAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims_MEAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims ■■■ Premium Amount---------✅
SCALER_Previous_Claims_STD_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims_MAX_Premium_Amount ■■■ Premium Amount✅
SCALER_Feedback3 ■■■ Premium Amount---------------✅
SCALER_Health_Score ■■■ Premium Amount------------✅
SCALER_Health_Risk_S

,Feature,Target,Statistic Test,Test Statistic,P-Value,Verdict
1,SCALER_Annual_Income,Premium Amount,SpearmanR,-0.060743,0.000000e+00,There is Relationship
2,SCALER_Credit_Score,Premium Amount,SpearmanR,-0.037141,0.000000e+00,There is Relationship
3,SCALER_Growth1,Premium Amount,SpearmanR,-0.052737,0.000000e+00,There is Relationship
4,SCALER_Money_Handling_Level1,Premium Amount,SpearmanR,-0.047643,0.000000e+00,There is Relationship
6,SCALER_Money_Per_Head,Premium Amount,SpearmanR,-0.052062,0.000000e+00,There is Relationship
5,SCALER_Growth,Premium Amount,SpearmanR,-0.054155,0.000000e+00,There is Relationship
7,SCALER_Money_Handling_Level,Premium Amount,SpearmanR,-0.070907,0.000000e+00,There is Relationship
8,SCALER_Credit_by_Score,Premium Amount,SpearmanR,-0.057530,0.000000e+00,There is Relationship
10,SCALER_Determinstic,Premium Amount,SpearmanR,-0.055946,0.000000e+00,There is Relationship
9,SCALER_Feedback1,Premium Amount,SpearmanR,-0.052730,0.000000e+00,There is Relationship


In [8]:
df.isnull().sum()

Health Conscious Level                                      0
SCALER_Annual_Income                                        0
SCALER_Credit_Score                                         0
SCALER_Growth1                                              0
SCALER_Money_Handling_Level1                                0
SCALER_Growth                                               0
SCALER_Money_Per_Head                                       0
SCALER_Money_Handling_Level                                 0
SCALER_Credit_by_Score                                      0
SCALER_Feedback1                                            0
SCALER_Determinstic                                         0
SCALER_Previous_Claims_MEDIAN_Premium_Amount                0
SCALER_Previous_Claims_MEAN_Premium_Amount                  0
SCALER_Previous_Claims                                      0
SCALER_Previous_Claims_STD_Premium_Amount                   9
SCALER_Previous_Claims_MAX_Premium_Amount                   0
SCALER_F

#
---
#

# Removing non related columns

In [9]:
# df.drop(columns=["SCALER_ENCODED_Marital_Status_Married", "SCALER_ENCODED_Customer_Feedback", "SCALER_PC2_Meaningless_df", "SCALER_PC1_Meaningless_df", "SCALER_ENCODED_Occupation_Unemployed"], inplace=True)

#
---
#

# Spliting Data

In [10]:
train = df.iloc[:1200000, :]
test = df.iloc[1200000:, :]

train.shape, test.shape

((1200000, 50), (800000, 50))

In [11]:
X = train.drop(columns="Premium Amount")
Y = train["Premium Amount"]
Y_log = np.log1p(Y)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_validate, y_train, y_validate = train_test_split(X, Y, test_size=10000)

In [14]:
x_validate.shape

(10000, 49)

In [15]:
test.drop(columns="Premium Amount", inplace=True)

In [16]:
test.shape

(800000, 49)

In [17]:
x_validate.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_Growth1,SCALER_Money_Handling_Level1,SCALER_Growth,SCALER_Money_Per_Head,SCALER_Money_Handling_Level,SCALER_Credit_by_Score,SCALER_Feedback1,SCALER_Determinstic,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_Health_Age_Interaction,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Marital_Status_Single,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount,SCALER_Number_of_Dependents
273714,6,3.149044,-0.625000,2.010557,4.179325,3.902804,5.696410,2.368627,-0.237398,1.138217,1.942913,0.0,-1.0,-1.0,-1.0,1.0,-0.5,1.506713,-1.506713,-0.655723,-0.536558,-1.0,2.250048,0.000000,0.0,0.103876,1.300721,1.224647e-15,-1.0,-0.429599,0.481364,1.75,-1.000000,-0.557755,0.00,-0.65174,0.819894,0.333333,-0.683947,-0.333333,0.4,0.0,1.0,0.333333,1.0,0.608696,0.000000,-1.0,0.000000
173918,4,-0.479874,-0.142241,-0.406837,-0.409995,-0.321949,-0.439238,-0.491461,0.126829,-0.361618,-0.478538,-1.0,0.0,0.0,0.0,0.0,0.0,0.058480,-0.058480,-0.055013,-0.834271,1.0,0.696827,0.666667,0.8,0.106182,-0.079380,1.224647e-15,-1.0,-0.655226,0.000000,0.00,-1.571429,-1.483870,-2.75,0.00000,0.046335,-0.500000,0.316053,0.666667,-0.6,0.0,0.0,-0.666667,0.0,0.782609,-0.133238,1.0,-0.127755
389813,6,-0.624276,0.064655,-0.503030,-0.558045,-0.490060,-0.450069,-0.631885,0.282927,-0.487344,-0.568851,0.0,-1.0,-1.0,-1.0,1.0,-0.5,0.898290,-0.898290,0.030169,-0.586427,-1.0,1.835772,0.000000,0.0,0.651812,1.067964,7.347881e-16,-0.6,-0.429599,0.481364,1.75,-1.000000,0.458999,1.50,-0.65174,0.963618,0.333333,0.316053,0.666667,-0.6,0.0,0.0,0.333333,1.0,0.869565,-0.783022,-1.0,-0.127755


#
---
#

# Base Model Building

In [18]:
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_log_error

In [19]:
# model = XGBRegressor(verbosity=3)

In [20]:
# model.fit(x_train, np.log(y_train))

In [21]:
# prediction = model.predict(x_validate)

In [22]:
# prediction[prediction < 0] = 0

In [23]:
# root_mean_squared_log_error(y_validate, prediction)

In [24]:
# prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : model.predict(test)})
# prediction

In [25]:
# prediction.to_csv("Dileep's_Submission_Take3.csv", index=False)

#
---
#

# Hyper-tuning -- IT is taking too much time

In [25]:
# from sklearn.model_selection import RandomizedSearchCV

In [26]:
# esimator = XGBRegressor(n_jobs=-1, verbosity=3)

In [27]:
# param_grid = {
#     "n_estimators" : range(50, 1000),
#     "max_depth" : range(5, 30)
# }

In [28]:
# cv_model = RandomizedSearchCV(estimator=esimator, param_distributions=param_grid, verbose=3, n_jobs=-1, cv=2, n_iter=60)

In [29]:
# cv_model.fit(x_train, y_train)

#
---
#

# Optuna Hyper-parameter Tuning

In [36]:
def objective_lgbm(trial):
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 300, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
        'device': 'gpu',
        'gpu_platform_id': 1,
        'gpu_device_id': 0,
        'verbose': -1,
    }

    model = LGBMRegressor(**params)
    model.fit(x_train, y_train, 
              eval_set=[(x_validate, y_validate)], 
              eval_metric='rmse', 
              callbacks=[early_stopping(stopping_rounds=200)])
    preds = model.predict(x_validate)
    return root_mean_squared_error(y_validate, preds)

study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=50)
print("Best parameters for LightGBM:", study_lgbm.best_params)

[I 2024-12-26 11:59:51,141] A new study created in memory with name: no-name-c492b729-82fc-4b92-8d52-c7d9734e357b


Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[532]	valid_0's rmse: 859.308


[I 2024-12-26 12:00:14,329] Trial 0 finished with value: 859.3079475655072 and parameters: {'n_estimators': 623, 'learning_rate': 0.0948378771777915, 'num_leaves': 54, 'max_depth': 9, 'min_child_samples': 90, 'subsample': 0.9269873397091147, 'colsample_bytree': 0.7131768289311744, 'reg_alpha': 6.714518748061464, 'reg_lambda': 8.890336189804234}. Best is trial 0 with value: 859.3079475655072.


Training until validation scores don't improve for 200 rounds


[I 2024-12-26 12:00:30,746] Trial 1 finished with value: 859.240602532061 and parameters: {'n_estimators': 567, 'learning_rate': 0.17535606072665025, 'num_leaves': 142, 'max_depth': 10, 'min_child_samples': 66, 'subsample': 0.7795180289057186, 'colsample_bytree': 0.6545832520654826, 'reg_alpha': 0.18772418473450161, 'reg_lambda': 9.515764958486654}. Best is trial 1 with value: 859.240602532061.


...

Early stopping, best iteration is:
[119]	valid_0's rmse: 859.634
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[705]	valid_0's rmse: 858.296


[I 2024-12-26 12:17:20,414] Trial 48 finished with value: 858.2960297901783 and parameters: {'n_estimators': 1305, 'learning_rate': 0.057039638071732894, 'num_leaves': 134, 'max_depth': 11, 'min_child_samples': 94, 'subsample': 0.8818839454903746, 'colsample_bytree': 0.8404645920435989, 'reg_alpha': 2.2092334134699305, 'reg_lambda': 6.978035716970699}. Best is trial 48 with value: 858.2960297901783.


Training until validation scores don't improve for 200 rounds


[I 2024-12-26 12:17:38,694] Trial 49 finished with value: 859.2223213133948 and parameters: {'n_estimators': 1025, 'learning_rate': 0.07180274728802814, 'num_leaves': 136, 'max_depth': 10, 'min_child_samples': 94, 'subsample': 0.8699315906108797, 'colsample_bytree': 0.8386371845770688, 'reg_alpha': 1.3160337315516974, 'reg_lambda': 6.351591962149395}. Best is trial 48 with value: 858.2960297901783.


Early stopping, best iteration is:
[68]	valid_0's rmse: 859.222
Best parameters for LightGBM: {'n_estimators': 1305, 'learning_rate': 0.057039638071732894, 'num_leaves': 134, 'max_depth': 11, 'min_child_samples': 94, 'subsample': 0.8818839454903746, 'colsample_bytree': 0.8404645920435989, 'reg_alpha': 2.2092334134699305, 'reg_lambda': 6.978035716970699}


In [37]:
study_lgbm.best_params

{'n_estimators': 1305,
 'learning_rate': 0.057039638071732894,
 'num_leaves': 134,
 'max_depth': 11,
 'min_child_samples': 94,
 'subsample': 0.8818839454903746,
 'colsample_bytree': 0.8404645920435989,
 'reg_alpha': 2.2092334134699305,
 'reg_lambda': 6.978035716970699}

In [ ]:
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor'
    }

    model = XGBRegressor(**params)
    model.fit(x_train, y_train, 
              eval_set=[(x_validate, y_validate)],
              verbose=3)
    preds = model.predict(x_validate)
    return root_mean_squared_error(y_validate, preds)

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("Best parameters for XGBoost:", study_xgb.best_params)

[I 2024-12-26 12:29:29,979] A new study created in memory with name: no-name-6836c3a8-db30-4ff3-a93f-4ee2cb114af1


[0]	validation_0-rmse:869.70575
[3]	validation_0-rmse:867.28528
[6]	validation_0-rmse:865.61881
[9]	validation_0-rmse:864.31073
[12]	validation_0-rmse:863.39579
[15]	validation_0-rmse:862.73961
[18]	validation_0-rmse:862.19855
[21]	validation_0-rmse:861.81017
[24]	validation_0-rmse:861.50998
[27]	validation_0-rmse:861.31032
[30]	validation_0-rmse:861.14328
[33]	validation_0-rmse:861.05722
[36]	validation_0-rmse:860.88299
[39]	validation_0-rmse:860.77856
[42]	validation_0-rmse:860.78165
[45]	validation_0-rmse:860.77354
[48]	validation_0-rmse:860.80008
[51]	validation_0-rmse:860.78755
[54]	validation_0-rmse:860.74404
[57]	validation_0-rmse:860.71809
[60]	validation_0-rmse:860.66687
[63]	validation_0-rmse:860.62122
[66]	validation_0-rmse:860.52872
[69]	validation_0-rmse:860.53815
[72]	validation_0-rmse:860.50359
[75]	validation_0-rmse:860.50461
[78]	validation_0-rmse:860.47813
[81]	validation_0-rmse:860.46864
[84]	validation_0-rmse:860.45612
[87]	validation_0-rmse:860.42075
[90]	validatio

#
---
#

# Let's do Cross-Validation

In [30]:
X["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [31]:
test["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [32]:
lgb_params = {
    'n_estimators': 1305,
     'learning_rate': 0.057039638071732894,
     'num_leaves': 134,
     'max_depth': 11,
     'min_child_samples': 94,
     'subsample': 0.8818839454903746,
     'colsample_bytree': 0.8404645920435989,
     'reg_alpha': 2.2092334134699305,
     'reg_lambda': 6.978035716970699
 }

# cat_params = {
#     'iterations': 3000, 
#     'learning_rate': 0.038365175314273574, 
#     'depth': 11, 
#     'l2_leaf_reg': 3.596285147607088, 
#     'bagging_temperature': 0.2618728648567565
# }

xgb_params = {
    'n_estimators': 1078, 
    'learning_rate': 0.016084079332671603, 
    'max_depth': 10, 
    'min_child_weight': 8, 
    'subsample': 0.8732132237392727, 
    'colsample_bytree': 0.9756972730817159, 
    'reg_alpha': 3.386299962300141, 
    'reg_lambda': 8.964009483088061
}




In [33]:
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score


splitter = KFold(n_splits=10)
models = []


count = 1

folds_df = pd.DataFrame()

for train_index, test_index in splitter.split(X, Y_log):
    fold_xtrain  = X.iloc[train_index, :]
    fold_ytrain  = Y_log.iloc[train_index]
    fold_xtest  = X.iloc[test_index, :]
    fold_ytest  = Y_log.iloc[test_index]

    # LightGBM model
    lgbm_model = LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=1, gpu_device_id=0, verbose=3)

    # CatBoost model
    # cat_model = CatBoostRegressor(**cat_params, verbose=3, task_type= 'GPU')

    # XGBoost model
    xgb_model = XGBRegressor(**xgb_params, verbose=3, tree_method="gpu_hist", predictor= 'gpu_predictor')

    # Voting Regressor
    model = VotingRegressor(
        estimators=[
            ('lgbm', lgbm_model),
            # ('cat', cat_model),
            ('xgb', xgb_model)
        ]
    )

    model.fit(fold_xtrain, fold_ytrain)
    fold_pred = np.maximum(0, model.predict(fold_xtest))
    fold_rmsle = root_mean_squared_log_error(np.expm1(fold_ytest), np.expm1(fold_pred))
    print(f"Fold {count} RMSLE: {fold_rmsle}")
    models.append(model)

    folds_df[f"Fold {i}"] = model.predict(test)
    count += 1

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.660050
[LightGBM] [Info] Total Bins 3810
[LightGBM] [Info] Number of data points in the train set: 1080000, number of used features: 43
[LightGBM] [Info] Using requested OpenCL platform 1 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 40 dense feature groups (41.20 MB) transferred to GPU in 0.064681 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 6.590257
[LightGBM] [Debug] Trained a tree with leaves = 79 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 79 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 7

In [34]:
# splitter = KFold(n_splits=10)

In [35]:
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score
# gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=150)
# xg_model = XGBRegressor(verbose=3, loss="absolute_error", learning_rate=0.075, n_estimators=100)

# count = 1

# folds_df = pd.DataFrame()

# for train_index, test_index in splitter.split(X, Y_log):
#     fold_xtrain  = X.iloc[train_index, :]
#     fold_ytrain  = Y_log.iloc[train_index]
#     fold_xtest  = X.iloc[test_index, :]
#     fold_ytest  = Y_log.iloc[test_index]

#     # print(fold_xtrain.shape, fold_ytrain.shape, fold_xtest.shape, fold_ytest.shape)

#     fold_ypred = xg_model.fit(fold_xtrain, fold_ytrain).predict(fold_xtest)
#     print(f"The Root mean squared log error for FOLD - {count} is --> {root_mean_squared_log_error(fold_ytest, fold_ypred)}")
    
#     folds_df[f"Fold {count}"] = xg_model.predict(test)
#     count += 1

In [36]:
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score

print(f"The Root mean squared log error is --> {np.exp(root_mean_squared_log_error(fold_ytest, fold_ypred))}")
print(f"The R-Square is --> {r2_score(fold_ytest, fold_ypred)}")


NameError: name 'fold_ypred' is not defined

In [47]:
folds_df.to_csv("folds_prediction.csv", index=False)

#
---
#

In [48]:
preds = pd.read_csv("folds_prediction.csv")
preds

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10
0,6.715447,6.688912,6.758556,6.776663,6.711772,6.727600,6.721043,6.731705,6.700436,6.710533
1,6.729539,6.715304,6.774331,6.727609,6.798041,6.781332,6.788819,6.792028,6.802070,6.834750
2,6.655867,6.660272,6.645963,6.672046,6.648002,6.639626,6.645034,6.668981,6.637800,6.648517
3,6.634870,6.652381,6.656625,6.623518,6.658536,6.634804,6.623298,6.632634,6.664675,6.654606
4,6.616711,6.623666,6.604294,6.602559,6.607086,6.612793,6.605227,6.628029,6.609004,6.628419
...,...,...,...,...,...,...,...,...,...,...
799995,6.811942,6.850738,6.841133,6.859432,6.850626,6.836756,6.854081,6.819692,6.855243,6.825788
799996,6.611531,6.641240,6.649746,6.680002,6.663827,6.621181,6.692587,6.688931,6.657662,6.622571
799997,6.756113,6.721759,6.795280,6.703122,6.799648,6.757790,6.718507,6.724326,6.764277,6.705512
799998,6.717124,6.720974,6.730668,6.721966,6.704302,6.736166,6.735196,6.715274,6.732140,6.736230


In [58]:
pred = preds.mean(axis=1).apply(np.expm1)

In [59]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : pred})
prediction

,id,Premium Amount
0,1200000,802.447562
1,1200001,823.934310
2,1200002,762.413289
3,1200003,751.422524
4,1200004,735.978383
...,...,...
799995,1999995,907.633506
799996,1999996,742.620672
799997,1999997,813.946144
799998,1999998,814.908071


In [60]:
prediction.to_csv("Dileep's_Submission_Take25.csv", index=False)

In [60]:
# Take 20 did after Feature engineering
# Before -- 821 Rank
# After -- 757

#
---
#

# Lets implement... `STACKING`

In [106]:
X.fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [107]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_log_error

In [111]:
lir_model = LinearRegression()
knn_model = KNeighborsRegressor()
rf_model = RandomForestRegressor(verbose=3, n_estimators=40)
gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=250)
xgb_model = XGBRegressor(verbose=3)
ridge_model = Ridge()
if_model = IsolationForest(verbose=3)

In [112]:
stacked_model = StackingRegressor(estimators=[
                    ("rf_model", rf_model),
                    # ("knn_model", knn_model),
                    ("xgb_model", xgb_model),
                    ("gb_model", gb_model),
                    # ("lir_model", lir_model)
                ], final_estimator=lir_model, verbose=3)

In [113]:
gb_model.fit(X, np.log(Y))

      Iter       Train Loss   Remaining Time 
         1           0.7717           34.10m
         2           0.7642           32.46m
         3           0.7623           30.65m
         4           0.7610           30.38m
         5           0.7604           30.08m
         6           0.7600           29.70m
         7           0.7598           29.28m
         8           0.7594           29.04m
         9           0.7592           28.73m
        10           0.7591           28.47m
        11           0.7591           28.43m
        12           0.7590           28.31m
        13           0.7589           28.50m
        14           0.7589           28.71m
        15           0.7588           28.62m
        16           0.7587           28.36m
        17           0.7587           28.18m
        18           0.7586           28.15m
        19           0.7584           27.90m
        20           0.7582           28.12m
        21           0.7582           28.06m
        2

GradientBoostingRegressor(learning_rate=0.5, loss='absolute_error',
                          n_estimators=250, verbose=3)

In [114]:
prediction = gb_model.predict(x_validate)

In [115]:
# prediction[prediction < 0] = prediction.mean()

In [116]:
root_mean_squared_log_error(y_validate, prediction)

np.float64(4.67300233947815)

In [122]:
test.fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [123]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : np.exp(gb_model.predict(test))})
prediction

,id,Premium Amount
0,1200000,957.075734
1,1200001,920.077180
2,1200002,905.662536
3,1200003,905.194894
4,1200004,833.240238
...,...,...
799995,1999995,1036.266459
799996,1999996,903.047919
799997,1999997,1005.565005
799998,1999998,935.444136


In [124]:
prediction.to_csv("Dileep's_Submission_Take14.csv", index=False)